# Convolutional Neural Net 

This is the forth introduction on Tensorflow for RVL224 students

In this section you will learn:

* How to build a ConvNet to classify images
* How to save and restore trained parameters in neural net


In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [2]:
mnist  = input_data.read_data_sets('/tmp/data/', one_hot=True) #load dataset
train_img   = mnist.train.images
train_label = mnist.train.labels
test_img    = mnist.test.images
test_label  = mnist.test.labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
learning_rate = 0.001
training_iters = 200
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_classes])
keep_prob = tf.placeholder('float') #dropout (keep probability)

In [16]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [17]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [23]:
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()

In [25]:
display_step = 10

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(training_iters): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1. })
        feeds = {x: batch_xs, y: batch_ys, keep_prob: 1.}
        cost_value = sess.run(cost, feed_dict=feeds)
    
    
    # DISPLAY
        if i % display_step == 0:
            feeds_train = {x: batch_xs, y: batch_ys, keep_prob: 1. }
            feeds_test = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1. }
            train_acc = sess.run(accr, feed_dict=feeds_train)
            test_acc = sess.run(accr, feed_dict=feeds_test)
            print ("step: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                % (i, training_iters, cost_value, train_acc, test_acc))
    
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

    
print ("DONE")


step: 000/200 cost: 65787.656250000 train_acc: 0.062 test_acc: 0.081
step: 010/200 cost: 22077.402343750 train_acc: 0.320 test_acc: 0.230
step: 020/200 cost: 11352.830078125 train_acc: 0.484 test_acc: 0.501
step: 030/200 cost: 7692.818847656 train_acc: 0.648 test_acc: 0.636
step: 040/200 cost: 4532.322265625 train_acc: 0.711 test_acc: 0.726
step: 050/200 cost: 3397.919921875 train_acc: 0.727 test_acc: 0.772
step: 060/200 cost: 3862.547851562 train_acc: 0.727 test_acc: 0.798
step: 070/200 cost: 971.943847656 train_acc: 0.906 test_acc: 0.828
step: 080/200 cost: 2036.701416016 train_acc: 0.859 test_acc: 0.842
step: 090/200 cost: 2412.441894531 train_acc: 0.867 test_acc: 0.848
step: 100/200 cost: 2633.680664062 train_acc: 0.828 test_acc: 0.856
step: 110/200 cost: 3356.569335938 train_acc: 0.820 test_acc: 0.866
step: 120/200 cost: 1631.283691406 train_acc: 0.867 test_acc: 0.865
step: 130/200 cost: 1812.924804688 train_acc: 0.875 test_acc: 0.872
step: 140/200 cost: 1805.681030273 train_acc: 

In [ ]:
with tf.Session() as sess:
  # Restore variables from disk.
   saver.restore(sess, "/tmp/model.ckpt")
   print("Model restored.")
